In [1]:
import plotly
import plotly.graph_objects as go
import urllib, json
import pandas as pd
import psycopg2
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()
conn = psycopg2.connect(f"dbname={os.getenv('SQL_DATABASE')} user={os.getenv('SQL_DATABASE')} password={os.getenv('SQL_PASSWORD')} host={os.getenv('SQL_HOST')} port={os.getenv('SQL_PORT')}")
from sqlalchemy import create_engine

engine = create_engine(f"postgresql+psycopg2://{os.getenv('SQL_USER')}:{os.getenv('SQL_PASSWORD')}@{os.getenv('SQL_HOST')}:{os.getenv('SQL_PORT')}/{os.getenv('SQL_DATABASE')}")


In [22]:
df_worflow_data = pd.read_sql("select * from analytics.flow_detections_change_status_30169_ze", con=conn)
df_worflow_data['new_status_code'] = df_worflow_data['new_status'].apply(lambda x : ''.join([ y[0] for y in x.replace('.','_').split('_')]) if not pd.isna(x) else '')
df_worflow_data['old_status_code'] = df_worflow_data['old_status'].apply(lambda x : ''.join([ y[0] for y in x.replace('.','_').split('_')]))
df_worflow_data['transition_code'] = df_worflow_data['old_status_code'] + '__' + df_worflow_data['new_status_code']

df_worflow_data

C:\Users\Raphael\AppData\Local\Temp\ipykernel_17284\1203924620.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_worflow_data = pd.read_sql("select * from analytics.flow_detections_change_status_30169_ze", con=conn)


,detection_id,detection_data_id,old_status,new_status,change_type,history_date,parcel_id,new_status_code,old_status_code,transition_code
0,6601009,6601007,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,None,leaf,2025-03-26 13:01:41.495894+01:00,1499824,,DNVNCNP,DNVNCNP__
1,6703684,6703682,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,None,leaf,2025-03-26 16:46:01.883695+01:00,1440085,,DNVNCNP,DNVNCNP__
2,6828825,6828823,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,None,leaf,2025-03-26 21:08:12.024294+01:00,2301022,,DNVNCNP,DNVNCNP__
3,6817525,6817523,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,None,leaf,2025-03-26 20:48:51.794110+01:00,1307735,,DNVNCNP,DNVNCNP__
4,6549678,6549676,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED,None,leaf,2025-03-26 10:52:35.295160+01:00,1450625,,DNVNCP,DNVNCP__
...,...,...,...,...,...,...,...,...,...,...
1680,6285330,6285328,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,None,leaf,2025-03-26 00:35:12.490585+01:00,2324893,,DNVNCNP,DNVNCNP__
1681,6258153,6258151,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED,None,leaf,2025-03-25 23:31:57.840845+01:00,2383235,,DNVNCP,DNVNCP__
1682,6729178,6729176,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,None,leaf,2025-03-26 17:51:58.670198+01:00,2382703,,DNVNCNP,DNVNCNP__
1683,6563623,6563621,SUSPECT.OFFICIAL_REPORT_DRAWN_UP.NOT_PRESCRIPTED,None,leaf,2025-06-30 09:11:04.101754+02:00,2294037,,SORDUNP,SORDUNP__


In [51]:
flow_status = df_worflow_data.groupby(['transition_code','new_status_code','old_status_code','old_status']).size().reset_index()
flow_status.rename(columns={0:'nb_transitions'}, inplace=True)
flow_status['color'] = flow_status['transition_code'].apply(lambda x : f"rgba({np.random.choice(range(256))}, {np.random.choice(range(256))}, {np.random.choice(range(256))}, 0.8)")
flow_status

,transition_code,new_status_code,old_status_code,old_status,nb_transitions,color
0,DNVCFNP__,,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...,14,"rgba(139, 86, 13, 0.8)"
1,DNVCFNP__SCFNP,SCFNP,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...,8,"rgba(134, 234, 220, 0.8)"
2,DNVCFP__SCFP,SCFP,DNVCFP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.PRESCRIBED,1,"rgba(224, 227, 52, 0.8)"
3,DNVNCNP__,,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,764,"rgba(14, 232, 125, 0.8)"
4,DNVNCNP__INCNP,INCNP,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,8,"rgba(18, 124, 146, 0.8)"
5,DNVNCNP__LNCNP,LNCNP,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,10,"rgba(61, 193, 222, 0.8)"
6,DNVNCNP__SNCNP,SNCNP,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,10,"rgba(141, 31, 195, 0.8)"
7,DNVNCP__,,DNVNCP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED,344,"rgba(163, 121, 102, 0.8)"
8,DNVNCP__SNCP,SNCP,DNVNCP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED,14,"rgba(67, 46, 149, 0.8)"
9,DNVORDUNP__,,DNVORDUNP,DETECTED_NOT_VERIFIED.OFFICIAL_REPORT_DRAWN_UP...,56,"rgba(253, 32, 107, 0.8)"


,old_status_code,old_status
0,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...
1,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...
2,DNVCFP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.PRESCRIBED
3,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...
4,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...
5,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...
6,DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...
7,DNVNCP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED
8,DNVNCP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED
9,DNVORDUNP,DETECTED_NOT_VERIFIED.OFFICIAL_REPORT_DRAWN_UP...


In [55]:
df_nodes = pd.DataFrame(data=df_worflow_data['old_status_code'].drop_duplicates().tolist(), columns=['status_code'])
df_nodes['color'] = df_nodes['status_code'].apply(lambda x : f"rgba({np.random.choice(range(256))}, {np.random.choice(range(256))}, {np.random.choice(range(256))}, 0.8)")
df_nodes = df_nodes.merge(flow_status[['old_status_code','old_status']], left_on ='status_code', right_on ='old_status_code')
df_nodes['id'] = df_nodes.index.tolist()
df_nodes

,status_code,color,old_status_code,old_status,id
0,DNVNCNP,"rgba(90, 117, 106, 0.8)",DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,0
1,DNVNCNP,"rgba(90, 117, 106, 0.8)",DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,1
2,DNVNCNP,"rgba(90, 117, 106, 0.8)",DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,2
3,DNVNCNP,"rgba(90, 117, 106, 0.8)",DNVNCNP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.NOT_PRESC...,3
4,DNVNCP,"rgba(108, 141, 73, 0.8)",DNVNCP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED,4
5,DNVNCP,"rgba(108, 141, 73, 0.8)",DNVNCP,DETECTED_NOT_VERIFIED.NOT_CONTROLLED.PRESCRIBED,5
6,LNCNP,"rgba(76, 230, 217, 0.8)",LNCNP,LEGITIMATE.NOT_CONTROLLED.NOT_PRESCRIPTED,6
7,SNCNP,"rgba(183, 123, 76, 0.8)",SNCNP,SUSPECT.NOT_CONTROLLED.NOT_PRESCRIPTED,7
8,SNCNP,"rgba(183, 123, 76, 0.8)",SNCNP,SUSPECT.NOT_CONTROLLED.NOT_PRESCRIPTED,8
9,SNCNP,"rgba(183, 123, 76, 0.8)",SNCNP,SUSPECT.NOT_CONTROLLED.NOT_PRESCRIPTED,9


In [56]:
flow_status = flow_status.merge(df_nodes[['id','status_code']],how='left', left_on='new_status_code', right_on='status_code').rename(columns={'id':'target'}).drop(columns=['status_code'])
flow_status = flow_status.merge(df_nodes[['id','status_code']],how='left', left_on='old_status_code', right_on='status_code').rename(columns={'id':'source'}).drop(columns=['status_code'])
flow_status = flow_status[~(flow_status.target.isna())]
flow_status

,transition_code,new_status_code,old_status_code,old_status,nb_transitions,color,target,source
2,DNVCFNP__SCFNP,SCFNP,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...,8,"rgba(134, 234, 220, 0.8)",17.0,21
3,DNVCFNP__SCFNP,SCFNP,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...,8,"rgba(134, 234, 220, 0.8)",17.0,22
4,DNVCFNP__SCFNP,SCFNP,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...,8,"rgba(134, 234, 220, 0.8)",18.0,21
5,DNVCFNP__SCFNP,SCFNP,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...,8,"rgba(134, 234, 220, 0.8)",18.0,22
6,DNVCFNP__SCFNP,SCFNP,DNVCFNP,DETECTED_NOT_VERIFIED.CONTROLLED_FIELD.NOT_PRE...,8,"rgba(134, 234, 220, 0.8)",19.0,21
...,...,...,...,...,...,...,...,...
337,SORRNP__SORRP,SORRP,SORRNP,SUSPECT.OBSERVARTION_REPORT_REDACTED.NOT_PRESC...,2,"rgba(173, 76, 24, 0.8)",33.0,37
338,SORRNP__SRNP,SRNP,SORRNP,SUSPECT.OBSERVARTION_REPORT_REDACTED.NOT_PRESC...,3,"rgba(58, 100, 187, 0.8)",27.0,34
339,SORRNP__SRNP,SRNP,SORRNP,SUSPECT.OBSERVARTION_REPORT_REDACTED.NOT_PRESC...,3,"rgba(58, 100, 187, 0.8)",27.0,35
340,SORRNP__SRNP,SRNP,SORRNP,SUSPECT.OBSERVARTION_REPORT_REDACTED.NOT_PRESC...,3,"rgba(58, 100, 187, 0.8)",27.0,36


In [59]:
# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity


fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  df_nodes['old_status'].tolist(),
      color =  df_nodes['color'].tolist()
    ),
    # Add links
    link = dict(
      source =  flow_status.source.tolist(),
      target =  flow_status.target.tolist(),
      value =  flow_status.nb_transitions.tolist(),
      label =  flow_status.transition_code.tolist(),
      color =  flow_status.color.tolist()
))])

fig.update_layout(title_text="Workflow des changements etats Aigle",
                  font_size=10)
fig.show(height= 800, width= 1600)

In [ ]:
url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())


In [ ]:
# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
                                    for src in data['data'][0]['link']['source']]

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label'],
      color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
                  font_size=10)
fig.show()